**Recommending runing in Google Colab**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/your-username/your-repo/blob/main/notebooks/your-notebook.ipynb)


In [3]:
import os
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import Adam
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision import models, transforms, datasets
from torchvision.io import read_image

ValueError: module functions cannot set METH_CLASS or METH_STATIC

In [ ]:
!unzip ./data/test.zip

In [ ]:
!unzip ./data/training.zip

In [ ]:
class FacialKeypointsDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

        keypoint_columns = [
            'left_eye_center_x', 'left_eye_center_y',
            'right_eye_center_x', 'right_eye_center_y',
            'left_eye_inner_corner_x', 'left_eye_inner_corner_y',
            'left_eye_outer_corner_x', 'left_eye_outer_corner_y',
            'right_eye_inner_corner_x', 'right_eye_inner_corner_y',
            'right_eye_outer_corner_x', 'right_eye_outer_corner_y',
            'left_eyebrow_inner_end_x', 'left_eyebrow_inner_end_y',
            'left_eyebrow_outer_end_x', 'left_eyebrow_outer_end_y',
            'right_eyebrow_inner_end_x', 'right_eyebrow_inner_end_y',
            'right_eyebrow_outer_end_x', 'right_eyebrow_outer_end_y',
            'nose_tip_x', 'nose_tip_y',
            'mouth_left_corner_x', 'mouth_left_corner_y',
            'mouth_right_corner_x', 'mouth_right_corner_y',
            'mouth_center_top_lip_x', 'mouth_center_top_lip_y',
            'mouth_center_bottom_lip_x', 'mouth_center_bottom_lip_y'
        ]
        self.keypoint_columns = [col for col in keypoint_columns if col in self.annotations.columns]



    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, f'{idx}.png')
        if not os.path.exists(img_name):
             return None

        image = Image.open(img_name).convert("L")

        label = self.annotations.loc[idx, self.keypoint_columns].values.astype('float32')

        label[np.isnan(label)] = 0.0

        label = torch.tensor(label)

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
output_dir = 'generated_images'
os.makedirs(output_dir, exist_ok=True)

train_df = pd.read_csv('./training.csv')

for index, row in train_df.iterrows():
    image_pixels = row['Image'].split(' ')
    image_array = np.array([p for p in image_pixels if p != ''], dtype=np.uint8)

    try:
        img = Image.fromarray(image_array.reshape(96, 96), mode='L')
        filename = f'{output_dir}/{index}.png'
        img.save(filename)
    except ValueError as e:
        print(f"Skipping row {index} due to incorrect pixel data length: {e}")
        continue

print(f"Generated {len(train_df)} images in the '{output_dir}' directory.")

In [ ]:
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
csv_file='training.csv'
root_dir='generated_images'
dataset = FacialKeypointsDataset(csv_file, root_dir, transform=transform)

In [ ]:
valid_indices = [i for i in range(len(dataset)) if dataset[i] is not None]
valid_dataset = torch.utils.data.Subset(dataset, valid_indices)

In [ ]:
train_size = int(0.8 * len(valid_dataset))
val_size = len(valid_dataset) - train_size
train_set, val_set = random_split(valid_dataset, [train_size, val_size])

In [ ]:
train_loader = DataLoader(train_set, batch_size=128, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_set, batch_size=128, num_workers=4, pin_memory=True)

In [ ]:
class KeypointCNN(nn.Module):
    def __init__(self):
        super(KeypointCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(64, 128, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(128, 256, kernel_size=3),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Dropout(0.1)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 10 * 10, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 30)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

In [ ]:
device = torch.device('cpu')
model = KeypointCNN().to(device)
criterion = nn.MSELoss()
optimizer = Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5
best_val_loss = float('inf')
epoch = 0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    val_loss = 0.0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
    loss = running_loss / len(train_loader)
    val_loss = val_loss / len(val_loader)
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
    print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss:.4f}, Validation Loss: {val_loss:.4f}")

In [ ]:
model = KeypointCNN()
model.load_state_dict(torch.load("keypoints_model.pth"))
model.eval()
model.to(device)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5], std=[0.5])
])

In [ ]:
def evaluate_model(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    with torch.no_grad():
        for images, keypoints in dataloader:
            images, keypoints = images.to(device), keypoints.to(device)
            outputs = model(images)
            loss = criterion(outputs, keypoints.view(outputs.shape))
            total_loss += loss.item()
    return total_loss / len(dataloader)

In [ ]:
val_loss = evaluate_model(model, val_loader, criterion, device)
print(f"Validation MSE: {val_loss:.4f}")